<a href="https://colab.research.google.com/github/dzejkopjarosinski/ZespolR/blob/main/MLF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinance
!pip install yahoo_fin
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Jakich wskaźników potrzeba:
- Debt2Equity - Pokazuje jakie jest zaangażowanie długu danej spółki w finansowaniu jej działalności operacyjnej(Pożądane jak najmniejsze)
- ROE-Return on Equity to miara pokazująca jaka część net income jest wynikiem inwestycji w equity danej spółki.
- Current ratio - Wskaźnik mówiący o tym jaką płynność w aktywach obrotowych posiada firma na pokrycie krótkoterminowego długu.
- ROA - Return on Assets pokazuje jak efektywnie dana spółka używa swoich aktywów do generowania zysków.
- P/E ratio - Price to Earnings porównuje cene akcji do przychodów na jedną akcje (im mniejsze tym firma bardziej przewartościowana).\
- GPM - Gross Profit Margin mówi o tym jaką średnią marżę na swojej działalności ma dana firma. Nim jest ona większa tym lepiej

In [ ]:
import pandas as pd
from yahoofinance import BalanceSheet
from yahoofinancials import YahooFinancials
from yahoofinance import IncomeStatement
import pprint

In [ ]:
start_date = '2018-01-01'
end_date = '2022-01-01'

In [ ]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'FB', 'TSLA', 'BRK-A', 'JPM', 'JNJ', 'V', 'PG', 'NVDA', 'MA', 'UNH', 'HD', 'BAC', 'PYPL', 'DIS', 'XOM']

financials = {}
income_statements={}
for ticker in tickers:
    try:
        yahoo_financials = YahooFinancials(ticker)
        balance_sheet_data = yahoo_financials.get_financial_stmts('annual', 'balance')
        income_statement_data = yahoo_financials.get_financial_stmts('annual', 'income')
        income_statements[ticker] = income_statement_data['incomeStatementHistory'][ticker]
        financials[ticker] = balance_sheet_data['balanceSheetHistory'][ticker]
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")



In [ ]:
# income_statements

In [ ]:
# financials

In [ ]:
financial2 = {}

for tick in tickers:
  financial2[tick] = {}
  for i,single_date in enumerate(financials[tick]):
    datekeys = list(single_date.keys())
    datekey = datekeys[0] 
    # print("Adding date ", datekey, "For ", ticker)
    financial2[tick][datekey] = single_date[datekey]
    # print("Added")
  # print("Finished ," ,tick, "Results:",financial2[tick])
  print("Finished, ", tick)


In [ ]:
# financial2

In [ ]:
financial2['AAPL']['2021-09-30']

{'cashAndCashEquivalents': 34940000000.0,
 'stockholdersEquity': 63090000000.0,
 'machineryFurnitureEquipment': 78659000000.0,
 'properties': 0.0,
 'commonStockEquity': 63090000000.0,
 'investmentsAndAdvances': 127877000000.0,
 'availableForSaleSecurities': 127877000000.0,
 'landAndImprovements': 20041000000.0,
 'investmentinFinancialAssets': 127877000000.0,
 'currentDeferredRevenue': 7612000000.0,
 'totalCapitalization': 172196000000.0,
 'capitalStock': 57365000000.0,
 'otherInvestments': 127877000000.0,
 'accountsReceivable': 26278000000.0,
 'shareIssued': 16426786000.0,
 'payablesAndAccruedExpenses': 54763000000.0,
 'totalNonCurrentLiabilitiesNetMinorityInterest': 162431000000.0,
 'otherCurrentBorrowings': 9613000000.0,
 'cashFinancial': 17305000000.0,
 'longTermDebtAndCapitalLeaseObligation': 109106000000.0,
 'otherShortTermInvestments': 27699000000.0,
 'cashCashEquivalentsAndShortTermInvestments': 62639000000.0,
 'receivables': 51506000000.0,
 'inventory': 6580000000.0,
 'accumula

In [ ]:
financials_calculated ={}
for ticker in tickers:
  financials_calculated[ticker] = []
  for i,each in enumerate(financials[ticker]):
    datekeys = list(each.keys())
    datekey = datekeys[0] 
    # Profitability ratios
    emptydict={}
    financials_calculated[ticker].append(emptydict)
    financials_calculated[ticker][i][datekey] = {}
    try:
      financials_calculated[ticker][i][datekey]["asset_debt_coverage"] = each[datekey]["totalAssets"]/each[datekey]["totalDebt"]
    except Exception as e:
        print(f"Failed to calculate asset_debt_coverage {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["net_income"] = -1 * each[datekey]['gainsLossesNotAffectingRetainedEarnings'] + each[datekey]['retainedEarnings']
    except Exception as e:
        print(f"Failed to calculate net_incom {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["return_on_equity"] = financials_calculated[ticker][i][datekey]["net_income"]/(each[datekey]['stockholdersEquity']/each[datekey]['shareIssued'])
    except Exception as e:
        print(f"Failed to calculate return_on_equity {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["return_on_assets"] = financials_calculated[ticker][i][datekey]["net_income"]/each[datekey]['totalAssets']
    except Exception as e:
        print(f"Failed to calculate return_on_assets {datekey}: {e}")    

    try:
      financials_calculated[ticker][i][datekey]["total_revenue"] = each[datekey]['totalRevenue']
    except Exception as e:
        print(f"Failed to calculate total_revenue {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["profit_margin"] = financials_calculated[ticker][i][datekey]["net_income"] / financials_calculated[ticker][i][datekey]["total_revenue"]
    except Exception as e:
        print(f"Failed to calculate profit_margin {datekey}: {e}")

    # Liquidity ratios
    try:
      financials_calculated[ticker][i][datekey]["current_assets"] = each[datekey]['cashAndCashEquivalents'] + each[datekey]['accountsReceivable'] + each[datekey]['inventory']
    except Exception as e:
        print(f"Failed to calculate current_assets {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["current_liabilities"] = each[datekey]['currentLiabilities']
    except Exception as e:
        print(f"Failed to calculate current_liabilities {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["current_ratio"] = financials_calculated[ticker][i][datekey]["current_assets"] / financials_calculated[ticker][i][datekey]["current_liabilities"]
    except Exception as e:
        print(f"Failed to calculate current_ratio {datekey}: {e}")

    # Solvency ratios
    try:
      financials_calculated[ticker][i][datekey]["total_asset"] = each[datekey]['totalAssets']
    except Exception as e:
        print(f"Failed to calculate total_asset {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["total_liabilities"] = each[datekey]['totalLiabilitiesNetMinorityInterest']
    except Exception as e:
        print(f"Failed to calculate total_liabilities {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["debt_to_equity_ratio"] = financials_calculated[ticker][i][datekey]["total_liabilities"] / each[datekey]['commonStockEquity']
    except Exception as e:
        print(f"Failed to calculate debt_to_equity_ratio {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["debt_to_assets_ratio"] = financials_calculated[ticker][i][datekey]["total_liabilities"] / financials_calculated[ticker][i][datekey]["total_asset"]
    except Exception as e:
        print(f"Failed to calculate debt_to_assets_ratio {datekey}: {e}")


    # Valuation ratios
    try:
      financials_calculated[ticker][i][datekey]["market_cap"] = each[datekey]['marketCap']
    except Exception as e:
      print(f"Failed to calculate market_cap {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["enterprise_value"] = each[datekey]['totalEnterpriseValue']
    except Exception as e:
      print(f"Failed to calculate enterprise_value {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["ev_to_ebitda"] = financials_calculated[ticker][i][datekey]["enterprise_value"] / each[datekey]['ebitda']
    except Exception as e:
      print(f"Failed to calculate ev_to_ebitda {datekey}: {e}")
        

Failed to calculate total_revenue 2019-09-30: 'totalRevenue'
Failed to calculate profit_margin 2019-09-30: 'total_revenue'
Failed to calculate market_cap 2019-09-30: 'marketCap'
Failed to calculate enterprise_value 2019-09-30: 'totalEnterpriseValue'
Failed to calculate ev_to_ebitda 2019-09-30: 'enterprise_value'
Failed to calculate total_revenue 2020-09-30: 'totalRevenue'
Failed to calculate profit_margin 2020-09-30: 'total_revenue'
Failed to calculate market_cap 2020-09-30: 'marketCap'
Failed to calculate enterprise_value 2020-09-30: 'totalEnterpriseValue'
Failed to calculate ev_to_ebitda 2020-09-30: 'enterprise_value'
Failed to calculate total_revenue 2021-09-30: 'totalRevenue'
Failed to calculate profit_margin 2021-09-30: 'total_revenue'
Failed to calculate market_cap 2021-09-30: 'marketCap'
Failed to calculate enterprise_value 2021-09-30: 'totalEnterpriseValue'
Failed to calculate ev_to_ebitda 2021-09-30: 'enterprise_value'
Failed to calculate total_revenue 2022-09-30: 'totalRevenu

In [ ]:
financials_calculated

{'AAPL': [{'2019-09-30': {'asset_debt_coverage': 3.1330439530944867,
    'net_income': 46482000000.0,
    'return_on_equity': 9129630260.454424,
    'return_on_assets': 0.13731108721596616,
    'current_assets': 75876000000.0,
    'current_liabilities': 105718000000.0,
    'current_ratio': 0.7177207287311527,
    'total_asset': 338516000000.0,
    'total_liabilities': 248028000000.0,
    'debt_to_equity_ratio': 2.7410043320661304,
    'debt_to_assets_ratio': 0.7326921031797611}},
  {'2020-09-30': {'asset_debt_coverage': 2.880643210359671,
    'net_income': 15372000000.0,
    'return_on_equity': 3994043386.5838165,
    'return_on_assets': 0.04746085066442721,
    'current_assets': 58197000000.0,
    'current_liabilities': 105392000000.0,
    'current_ratio': 0.5521956125702141,
    'total_asset': 323888000000.0,
    'total_liabilities': 258549000000.0,
    'debt_to_equity_ratio': 3.957039440456695,
    'debt_to_assets_ratio': 0.7982666847799239}},
  {'2021-09-30': {'asset_debt_coverage'

In [ ]:
financials_calculated_nolist = {}

for tick in tickers:
  financials_calculated_nolist[tick] = {}
  for i,single_date in enumerate(financials_calculated[tick]):
    datekeys = list(single_date.keys())
    datekey = datekeys[0] 
    print("Adding date ", i)
    financials_calculated_nolist[tick][datekey] = single_date[datekey]
    print("Added")
  print("Finished ," ,tick, "Results:",financials_calculated_nolist[tick])


In [ ]:
financials_calculated_nolist

{'AAPL': {'2019-09-30': {'asset_debt_coverage': 3.1330439530944867,
   'net_income': 46482000000.0,
   'current_assets': 75876000000.0,
   'current_liabilities': 105718000000.0,
   'current_ratio': 0.7177207287311527,
   'total_asset': 338516000000.0,
   'total_liabilities': 248028000000.0,
   'debt_to_equity_ratio': 2.7410043320661304,
   'debt_to_assets_ratio': 0.7326921031797611},
  '2020-09-30': {'asset_debt_coverage': 2.880643210359671,
   'net_income': 15372000000.0,
   'current_assets': 58197000000.0,
   'current_liabilities': 105392000000.0,
   'current_ratio': 0.5521956125702141,
   'total_asset': 323888000000.0,
   'total_liabilities': 258549000000.0,
   'debt_to_equity_ratio': 3.957039440456695,
   'debt_to_assets_ratio': 0.7982666847799239},
  '2021-09-30': {'asset_debt_coverage': 2.8143426422598,
   'net_income': 5399000000.0,
   'current_assets': 67798000000.0,
   'current_liabilities': 125481000000.0,
   'current_ratio': 0.5403049067189455,
   'total_asset': 351002000000

In [ ]:
import yahoo_fin.stock_info as si
import pandas as pd

# Get data in the current column for each stock’s valuation table
tickers_stats = {}
for ticker in tickers:
  temp = si.get_stats_valuation(ticker)
  temp = temp.iloc[:,:2]
  temp.columns = ["Attribute", "Recent"]

  tickers_stats[ticker] = temp

  # combine all the stats valuation tables into a single data frame
  combined_stats = pd.concat(tickers_stats)
  combined_stats = combined_stats.reset_index()

  del combined_stats["level_1"]

  # update column names
  combined_stats.columns = ["Ticker", "Attribute", "Recent"]

In [ ]:
combined_stats[combined_stats['Ticker']=="AAPL"]

,Ticker,Attribute,Recent
0,AAPL,Market Cap (intraday),2.54T
1,AAPL,Enterprise Value,2.60T
2,AAPL,Trailing P/E,27.30
3,AAPL,Forward P/E,27.17
4,AAPL,PEG Ratio (5 yr expected),2.72
5,AAPL,Price/Sales (ttm),6.72
6,AAPL,Price/Book (mrq),44.85
7,AAPL,Enterprise Value/Revenue,6.72
8,AAPL,Enterprise Value/EBITDA,20.34


In [ ]:
# Profitability ratios
financials['AAPL']['net_income'] / financials['AAPL']['total_revenue']

# Liquidity ratios
financials['AAPL']['current_assets'] / financials['AAPL']['current_liabilities']

# Solvency ratios
financials['AAPL']['total_assets'] / financials['AAPL']['total_liabilities']

# Valuation ratios
financials['AAPL']['market_cap'] / financials['AAPL']['ebitda']


In [ ]:
rankings = {}
for ticker, data in financials.items():
    try:
        rankings[ticker] = data['net_income'] / data['total_revenue']
    except Exception as e:
        print(f"Failed to calculate ranking for {ticker}: {e}")

rankings = pd.DataFrame(rankings)
rankings = rankings.rank(axis=1, ascending=False)


Failed to calculate ranking for AAPL: list indices must be integers or slices, not str
Failed to calculate ranking for MSFT: list indices must be integers or slices, not str
Failed to calculate ranking for AMZN: list indices must be integers or slices, not str
Failed to calculate ranking for GOOG: list indices must be integers or slices, not str
Failed to calculate ranking for GOOGL: list indices must be integers or slices, not str
Failed to calculate ranking for FB: list indices must be integers or slices, not str
Failed to calculate ranking for TSLA: list indices must be integers or slices, not str
Failed to calculate ranking for BRK-A: list indices must be integers or slices, not str
Failed to calculate ranking for JPM: list indices must be integers or slices, not str
Failed to calculate ranking for JNJ: list indices must be integers or slices, not str
Failed to calculate ranking for V: list indices must be integers or slices, not str
Failed to calculate ranking for PG: list indices

In [ ]:
scores = rankings.sum()


In [ ]:
top_10 = scores.nlargest(10)


# Podstawowy Model

In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression

In [83]:
data = {
    ticker: {
        date: values
        for date, values in financials_calculated_nolist[ticker].items()
    }
    for ticker in financials_calculated_nolist.keys()
}

df = pd.DataFrame.from_dict({(i, j): data[i][j] for i in data.keys() for j in data[i].keys()}, orient='index')
df.index.names = ['ticker', 'date']

In [86]:
# Define list of companies to include in the index
companies = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB']

# Collect historical financial data for each company
data = df

# Calculate daily returns for each company
returns = data.pct_change().dropna()

In [87]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(returns)

# Reduce dimensionality with PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)

# Cluster the data with KMeans or Gaussian Mixture Models
kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(principal_components)

# Alternatively, you can use Gaussian Mixture Models
gmm = GaussianMixture(n_components=5)
clusters = gmm.fit_predict(principal_components)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [88]:
# Split the data into training and testing sets
train_data = returns.iloc[:-252]
test_data = returns.iloc[-252:]

# Train the model with Linear Regression
model = LinearRegression()
model.fit(train_data, clusters)


ValueError: ignored

In [ ]:
# Split the data into training and testing sets
train_data = returns.iloc[:-252]
test_data = returns.iloc[-252:]

# Train the model with Linear Regression
model = LinearRegression()
model.fit(train_data, clusters)


In [ ]:
# Allocate weights based on the model predictions
weights = model.predict(test_data)

# Normalize the weights to add up to 1
weights = weights / np.sum(weights)

# Print the weights for each company
for i, company in enumerate(companies):
    print(f'{company}: {weights[i]:.2%}')
